This script identifies bus stops within the isochrones

### 0. Libraries

In [ ]:
import geopandas as gpd
import pandas as pd


### 1. Bus stop
Identify the bus stop in each isochrone

In [53]:
# Load the shapefile of bus stops
bus_stop = gpd.read_file("D:/Climate_Shelter_Index/fermate_bus/tper-fermate-autobus.shp")
# Load the shapefile of isochrones
isochrones = gpd.read_file ("D:/Climate_Shelter_Index/isocrona/10min_isochrones.shp" )

In [54]:
isochrones

,min_dist,nome,geometry
0,10,AIUOLA VIA ALTOBELLI,"POLYGON ((11.27999 44.49796, 11.28415 44.50191..."
1,10,AIUOLE DANTI,"POLYGON ((11.30458 44.49247, 11.30429 44.49218..."
2,10,AIUOLE RAMENGHI,"POLYGON ((11.29943 44.49078, 11.29906 44.49065..."
3,10,AIUOLE AIMO,"POLYGON ((11.34802 44.52891, 11.34812 44.52797..."
4,10,AIUOLE COMPARTO BECCACCINO LA PIRA CAPITINI,"POLYGON ((11.29182 44.48931, 11.29157 44.48967..."
...,...,...,...
404,10,VERDE CONDOMINIO VIA GALEAZZA 14 - 28,"POLYGON ((11.27652 44.49596, 11.27467 44.49522..."
405,10,VERDE PASOLINI - ORTI (EX GIARDINO P. P. PASOL...,"POLYGON ((11.39728 44.51188, 11.39689 44.51179..."
406,10,VERDE SPONDA RUZZOLA,"POLYGON ((11.29340 44.50744, 11.29348 44.50738..."
407,10,VIALE FELSINA-VICOLO NERO,"POLYGON ((11.38389 44.49030, 11.38413 44.49031..."


In [55]:
unique_bus_stops = bus_stop.drop_duplicates(subset='codice', keep='first')

In [ ]:
# Perform a spatial join to check if each isochrones contains a bus stop
results_df = gpd.sjoin(isochrones[['min_dist','nome', 'geometry']], unique_bus_stops[['codice', 'geometry']], how="left", op="intersects")

# Create a new column 'bus_stop' indicating if there is a bus stop
results_df["bus_stop"] = results_df["codice"].notnull().replace({True: "yes", False: "no"})

In [57]:
# Create a new column 'id_busstop' with the ID of the bus stop if present
results_df["id_busstop"] = results_df["codice"].fillna("N/A")

In [58]:
results_df = results_df[['min_dist','nome', 'codice', 'bus_stop', 'id_busstop']]

In [59]:
results_df = results_df.merge(unique_bus_stops[['codice', 'geometry']], 
                              left_on='id_busstop', right_on='codice', how='left')

In [60]:
# Drop the duplicate  columns and rename 
results_df = results_df.drop(columns=[ 'codice_x', 'codice_y'])

In [61]:
results_df.head()

,min_dist,nome,bus_stop,id_busstop,geometry
0,10,AIUOLA VIA ALTOBELLI,yes,7140,POINT (11.28694 44.49412)
1,10,AIUOLA VIA ALTOBELLI,yes,7144,POINT (11.28452 44.49437)
2,10,AIUOLA VIA ALTOBELLI,yes,7143,POINT (11.28483 44.49477)
3,10,AIUOLA VIA ALTOBELLI,yes,7142,POINT (11.28478 44.49499)
4,10,AIUOLA VIA ALTOBELLI,yes,7162,POINT (11.28868 44.49530)


In [62]:
# Create a GeoDataFrame
results_gdf = gpd.GeoDataFrame(results_df, geometry='geometry', crs="epsg:4326")

In [63]:
# Save the GeoDataFrame as a shapefile
results_gdf.to_file('D:/Climate_Shelter_Index/isocrona/bus_stop_10_ga.shp')

### 2. Number of bus stop per isochrone

In [64]:
bus_stop_ga = gpd.read_file(r"D:\Climate_Shelter_Index\isocrona\bus_stop_10_ga.shp")

In [65]:
# Create a DataFrame to store the aggregation
sum_bus_stop = bus_stop_ga[['nome', 'min_dist']].copy()
# Count the occurrences of each 'nome'
sum_bus_stop['n_busstop'] = sum_bus_stop.groupby('nome')['nome'].transform('count')

sum_bus_stop.head()

,nome,min_dist,n_busstop
0,AIUOLA VIA ALTOBELLI,10,30
1,AIUOLA VIA ALTOBELLI,10,30
2,AIUOLA VIA ALTOBELLI,10,30
3,AIUOLA VIA ALTOBELLI,10,30
4,AIUOLA VIA ALTOBELLI,10,30


In [66]:
# Drop duplicates to keep only unique rows
sum_bus_stop = sum_bus_stop.drop_duplicates(subset=['nome', 'min_dist', 'n_busstop'])

In [67]:
sum_bus_stop.head()

,nome,min_dist,n_busstop
0,AIUOLA VIA ALTOBELLI,10,30
30,AIUOLE DANTI,10,15
45,AIUOLE RAMENGHI,10,20
65,AIUOLE AIMO,10,14
79,AIUOLE COMPARTO BECCACCINO LA PIRA CAPITINI,10,29


In [68]:
sum_bus_stop.to_csv('D:/Climate_Shelter_Index/isocrona/sum_bus_10_stop.csv', index=False)